In [ ]:
import gc
import toad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score,roc_curve,auc
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': 5,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47,
         }
import random
import os
import numpy as np
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(123)   

In [ ]:
train.head(5)

In [ ]:
y=train.pop('label')

In [ ]:
y.value_counts(1)

In [ ]:
X=train
y_test=test.pop('label')
X_test=test

In [ ]:
y_test.value_counts(1)

In [ ]:
for data in [X,X_test]:
    data.drop('CUST_ID',axis=1,inplace=True)

In [ ]:
NFOLDS = 5
folds = StratifiedKFold(n_splits=NFOLDS)
columns = X.columns
splits = folds.split(X, y)
y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X.shape[0])
score = 0

feature_importances = pd.DataFrame()
feature_importances['feature'] = columns
  
for fold_n, (train_index, valid_index) in enumerate(splits):
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)

    clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=200, early_stopping_rounds=500)
    
    feature_importances[f'fold_{fold_n + 1}'] = clf.feature_importance()
    
    y_pred_valid = clf.predict(X_valid)
    y_oof[valid_index] = y_pred_valid
    print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_valid, y_pred_valid)}")
    
    score += roc_auc_score(y_valid, y_pred_valid) / NFOLDS
    y_preds += clf.predict(X_test) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()
    
print(f"\nMean AUC = {score}")
print(f"Out of folds AUC = {roc_auc_score(y, y_oof)}")

In [ ]:
print(f"oot test Out of folds AUC = {roc_auc_score(y_test, y_preds)}")

In [ ]:
X.head(5)

In [ ]:
os=['OS_0',
 'OS_1',
 'OS_2',
 'OS_3',
 'OS_4',
 'OS_5',
 'OS_6',
 'OS_7',
 'OS_8',
 'OS_9',
 'OS_10',
 'OS_11',
 'OS_12',]
payment=[
 'Payment_1',
 'Payment_2',
 'Payment_3',
 'Payment_4',
 'Payment_5',
 'Payment_6',
 'Payment_7',
 'Payment_8',
 'Payment_9',
 'Payment_10',
 'Payment_11',
 'Payment_12',]
spend=[
 'Spend_1',
 'Spend_2',
 'Spend_3',
 'Spend_4',
 'Spend_5',
 'Spend_6',
 'Spend_7',
 'Spend_8',
 'Spend_9',
 'Spend_10',
 'Spend_11',
 'Spend_12',]

In [ ]:
os=train[os]
payment=train[payment]
spend=train[spend]

In [ ]:
payment.insert(0,'mask_payment',0)##序列长度不相等进行补0，后续要使用mask进行屏蔽
spend.insert(0,'mask_spend',0)

In [ ]:
t=pd.DataFrame()
t['os']=os.head(1).T[0].values
t['payment']=payment.head(1).T[0].values
t['spend']=spend.head(1).T[0].values

In [ ]:
t.values.reshape(-1,13,3)

In [ ]:
sequence_data=pd.concat([os,payment,spend],axis=1)

In [ ]:
os=['OS_0',
 'OS_1',
 'OS_2',
 'OS_3',
 'OS_4',
 'OS_5',
 'OS_6',
 'OS_7',
 'OS_8',
 'OS_9',
 'OS_10',
 'OS_11',
 'OS_12',]
payment=[
 'Payment_1',
 'Payment_2',
 'Payment_3',
 'Payment_4',
 'Payment_5',
 'Payment_6',
 'Payment_7',
 'Payment_8',
 'Payment_9',
 'Payment_10',
 'Payment_11',
 'Payment_12',]
spend=[
 'Spend_1',
 'Spend_2',
 'Spend_3',
 'Spend_4',
 'Spend_5',
 'Spend_6',
 'Spend_7',
 'Spend_8',
 'Spend_9',
 'Spend_10',
 'Spend_11',
 'Spend_12',]
os=test[os]
payment=test[payment]
spend=test[spend]
payment.insert(0,'mask_payment',0)##序列长度不相等进行补0，后续要使用mask进行屏蔽
spend.insert(0,'mask_spend',0)

sequence_data_test=pd.concat([os,payment,spend],axis=1)
sequence_data_test=sequence_data_test.values.reshape(12043,13,3,order='F')

In [ ]:
sequence_data.shape

In [ ]:
sequence_data=sequence_data.values.reshape(28099,13,3,order='F')

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
inputs1 = Input(shape=(13,3))
lstm1,state_h,state_c = LSTM(128,activation='tanh',return_sequences=True,return_state=True,recurrent_dropout=0.5)(inputs1)
outputs=Dense(1,activation='sigmoid')(state_h)
model = Model(inputs = inputs1,outputs = [outputs])
model.compile(loss='binary_crossentropy',
              optimizer='adam')

In [ ]:
model.fit(sequence_data, y, batch_size = 100, epochs = 50, verbose = 10)

In [ ]:
get_hidden_state=Model(inputs=inputs1,outputs=[lstm1,state_h,state_c])

In [ ]:
#提取隐藏层
results=get_hidden_state.predict(sequence_data)
results_test=get_hidden_state.predict(sequence_data_test)

In [ ]:
results[2]

In [ ]:
#与原始特征合并
times_feature=pd.DataFrame(results[2])
for col in times_feature.columns:
    X[col]=times_feature[col].values

In [ ]:
times_feature_test=pd.DataFrame(get_hidden_state.predict(sequence_data_test)[2])
for col in times_feature_test.columns:
    X_test[col]=times_feature_test[col].values

In [ ]:
NFOLDS = 5
folds = StratifiedKFold(n_splits=NFOLDS)
columns = X.columns
splits = folds.split(X, y)
y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X.shape[0])
score = 0

feature_importances = pd.DataFrame()
feature_importances['feature'] = columns
  
for fold_n, (train_index, valid_index) in enumerate(splits):
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)

    clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=200, early_stopping_rounds=500)
    
    feature_importances[f'fold_{fold_n + 1}'] = clf.feature_importance()
    
    y_pred_valid = clf.predict(X_valid)
    y_oof[valid_index] = y_pred_valid
    print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_valid, y_pred_valid)}")
    
    score += roc_auc_score(y_valid, y_pred_valid) / NFOLDS
    y_preds += clf.predict(X_test) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()
    
print(f"\nMean AUC = {score}")
print(f"Out of folds AUC = {roc_auc_score(y, y_oof)}")

In [ ]:
print(f"oot test Out of folds AUC = {roc_auc_score(y_test, y_preds)}")